In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
import random

In [2]:
# Set random seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)  # This sets all random seeds in keras
tf.config.experimental.enable_op_determinism()  # For complete reproducibility

In [3]:
def prepare_structure_data(df):
    """Structure data preparation without contacts"""
    features_list = []
    middle_pos = 16  
    
    # Normalize angles to their circular nature
    def normalize_angles(angle_array, pos):
        angles = np.array([arr[pos] for arr in angle_array])
        angle_rad = np.pi * angles / 180.0
        return np.stack([np.sin(angle_rad), np.cos(angle_rad)], axis=-1)
    
    # 1. Process angles
    angles = ['phi', 'psi', 'omega']
    for angle in angles:
        angle_arrays = np.array([np.array(eval(x)) for x in df[angle]])
        angle_features = normalize_angles(angle_arrays, middle_pos)
        features_list.append(angle_features)
        print(f"{angle} features shape: {angle_features.shape}")
    
    # 2. Process SASA
    sasa_arrays = np.array([np.array(eval(x)) for x in df['sasa']])
    scaler = RobustScaler()
    sasa_features = []
    for pos in [middle_pos-1, middle_pos, middle_pos+1]:
        sasa_pos = np.array([arr[pos] for arr in sasa_arrays]).reshape(-1, 1)
        sasa_scaled = scaler.fit_transform(sasa_pos)
        sasa_features.append(sasa_scaled)
    sasa_features = np.concatenate(sasa_features, axis=1)
    features_list.append(sasa_features)
    print(f"SASA features shape: {sasa_features.shape}")
    
    # 3. Process chi angles
    chi_angles = ['chi1', 'chi2', 'chi3', 'chi4']
    for chi in chi_angles:
        chi_arrays = np.array([np.array(eval(x)) for x in df[chi]])
        chi_features = normalize_angles(chi_arrays, middle_pos)
        features_list.append(chi_features)
        print(f"{chi} features shape: {chi_features.shape}")
    
    # 4. Process SS (optional)
    ss_arrays = np.array([list(seq) for seq in df['ss']])
    ss_center = ss_arrays[:, middle_pos]
    ss_encoded = np.zeros((len(ss_arrays), 3))
    ss_map = {'H': 0, 'E': 1, 'L': 2}
    for i, ss in enumerate(ss_center):
        ss_encoded[i, ss_map[ss]] = 1
    features_list.append(ss_encoded)
    print(f"SS features shape: {ss_encoded.shape}")
    
    # 5. Process plDDT
    plddt_arrays = np.array([np.array(eval(x)) for x in df['plDDT']])
    plddt_center = np.array([arr[middle_pos] for arr in plddt_arrays]).reshape(-1, 1)
    scaler = RobustScaler()
    plddt_scaled = scaler.fit_transform(plddt_center)
    features_list.append(plddt_scaled)
    print(f"plDDT features shape: {plddt_scaled.shape}")
    
    # Combine all features
    features = np.concatenate(features_list, axis=1)
    print(f"\nFinal combined features shape: {features.shape}")
    print("Feature list lengths:", [f.shape[1] for f in features_list])
    
    return features

In [4]:
def prepare_contact_map(df, threshold=8.0):
    """
    Process distance maps into binary contact maps with a specific threshold
    
    Args:
        df: DataFrame containing distance_map column
        threshold: Distance threshold to consider a contact (in Angstroms)
        
    Returns:
        Array of binary contact maps with shape (n_samples, 33, 33)
    """
    contact_maps = []
    
    for map_str in df['distance_map']:
        # Convert string to array and reshape
        values = np.array([float(x) for x in map_str.split(',')])
        distance_map = values.reshape(33, 33)
        
        # Convert to binary contact map based on threshold
        contact_map = np.zeros_like(distance_map)
        contact_map[distance_map <= threshold] = 1
        contact_map[distance_map == 0] = 0  # No self-contacts or missing values
        
        contact_maps.append(contact_map)
    
    return np.array(contact_maps)

In [5]:
def create_combined_structure_model(struct_input_dim, seq_length=33):
    """
    Create a model that combines contact map data with other structural features
    
    Args:
        struct_input_dim: The dimensionality of the structural features
        seq_length: Sequence length (default 33)
        
    Returns:
        A Keras model
    """
    # Input for structural features
    struct_input = tf.keras.layers.Input(shape=(struct_input_dim,), name='structure_input')
    x_struct = tf.keras.layers.Dense(64, activation='relu')(struct_input)
    x_struct = tf.keras.layers.BatchNormalization()(x_struct)
    x_struct = tf.keras.layers.Dropout(0.3)(x_struct)
    x_struct = tf.keras.layers.Dense(32, activation='relu')(x_struct)
    
    # Input and processing for contact map
    contact_input = tf.keras.layers.Input(shape=(seq_length, seq_length), name='contact_map')
    x_contact = tf.keras.layers.Reshape((seq_length, seq_length, 1))(contact_input)
    x_contact = tf.keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same')(x_contact)
    x_contact = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x_contact)
    x_contact = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same')(x_contact)
    x_contact = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x_contact)
    x_contact = tf.keras.layers.Flatten()(x_contact)
    x_contact = tf.keras.layers.Dense(32, activation='relu')(x_contact)
    
    # Combine both feature sets
    combined = tf.keras.layers.Concatenate()([x_struct, x_contact])
    
    # Final layers
    x = tf.keras.layers.BatchNormalization()(combined)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    # Create model
    model = tf.keras.Model(inputs=[struct_input, contact_input], outputs=outputs)
    
    return model

In [6]:
def print_metrics(y_true, y_pred):
    """
    Print comprehensive evaluation metrics
    
    Parameters:
    y_true: array-like of true labels
    y_pred: array-like of predicted labels
    """
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    # Calculate metrics
    acc = accuracy_score(y_true, y_pred)
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    sensitivity = cm[1][1] / (cm[1][1] + cm[1][0])  # True Positive Rate
    specificity = cm[0][0] / (cm[0][0] + cm[0][1])  # True Negative Rate
    
    # Print results
    print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Sensitivity: {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print("Confusion Matrix:")
    print(cm)

In [ ]:
def train_and_evaluate_combined_structure(threshold=8.0):
    """Train and evaluate combined structure and contact map model"""
    print("Loading structural data...")
    
    # Load data
    train_df = pd.read_csv("../../../../data/train/structure/processed_features_train.csv")
    test_df = pd.read_csv("../../../../data/test/structure/processed_features_test.csv")
    
    # Extract labels
    y_train = train_df['label'].values
    y_test = test_df['label'].values
    
    # Prepare feature data
    X_train_struct = prepare_structure_data(train_df)
    X_test_struct = prepare_structure_data(test_df)
    
    # Prepare contact map data
    X_train_contact = prepare_contact_map(train_df, threshold=threshold)
    X_test_contact = prepare_contact_map(test_df, threshold=threshold)
    
    # Shuffle training data
    shuffle_idx = np.random.RandomState(42).permutation(len(y_train))
    X_train_struct = X_train_struct[shuffle_idx]
    X_train_contact = X_train_contact[shuffle_idx]
    y_train = y_train[shuffle_idx]
    
    # Calculate class weights
    total_samples = len(y_train)
    pos_samples = np.sum(y_train == 1)
    neg_samples = np.sum(y_train == 0)
    
    class_weights = {
        0: total_samples / (2 * neg_samples),
        1: total_samples / (2 * pos_samples)
    }
    
    print(f"Training data shapes - Structural: {X_train_struct.shape}, Contact Map: {X_train_contact.shape}")
    print(f"Testing data shapes - Structural: {X_test_struct.shape}, Contact Map: {X_test_contact.shape}")
    print(f"Class weights: {class_weights}")
    
    # Cross-validation setup
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    metrics = {'acc': [], 'balanced_acc': [], 'mcc': [], 'sn': [], 'sp': []}
    test_predictions = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_struct, y_train), 1):
        print(f"\nFold {fold}/5")
        
        # Callbacks
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-6
            )
        ]
        
        # Create and compile model
        model = create_combined_structure_model(X_train_struct.shape[1])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        # Train model
        history = model.fit(
            [X_train_struct[train_idx], X_train_contact[train_idx]], 
            y_train[train_idx],
            validation_data=(
                [X_train_struct[val_idx], X_train_contact[val_idx]], 
                y_train[val_idx]
            ),
            batch_size=32,
            epochs=50,
            callbacks=callbacks,
            class_weight=class_weights,
            verbose=1
        )
        
        # Evaluate on validation set
        val_pred = model.predict([X_train_struct[val_idx], X_train_contact[val_idx]])
        val_pred_binary = (val_pred > 0.5).astype(int).flatten()
        
        # Calculate metrics
        cm = confusion_matrix(y_train[val_idx], val_pred_binary)
        metrics['acc'].append(accuracy_score(y_train[val_idx], val_pred_binary))
        metrics['balanced_acc'].append(balanced_accuracy_score(y_train[val_idx], val_pred_binary))
        metrics['mcc'].append(matthews_corrcoef(y_train[val_idx], val_pred_binary))
        metrics['sn'].append(cm[1][1]/(cm[1][1]+cm[1][0]) if cm[1][0]+cm[1][1] > 0 else 0)
        metrics['sp'].append(cm[0][0]/(cm[0][0]+cm[0][1]) if cm[0][0]+cm[0][1] > 0 else 0)
        
        # Predict on test set
        test_pred = model.predict([X_test_struct, X_test_contact])
        test_predictions.append(test_pred)
        
        print(f"\nFold {fold} Results:")
        print(f"Accuracy: {metrics['acc'][-1]:.4f}")
        print(f"Balanced Accuracy: {metrics['balanced_acc'][-1]:.4f}")
        print(f"MCC: {metrics['mcc'][-1]:.4f}")
        print(f"Sensitivity: {metrics['sn'][-1]:.4f}")
        print(f"Specificity: {metrics['sp'][-1]:.4f}")
    
    # Print average cross-validation results
    print("\nAverage Cross-validation Results:")
    for metric in metrics:
        print(f"{metric.upper()}: {np.mean(metrics[metric]):.4f} ± {np.std(metrics[metric]):.4f}")
    
    # Ensemble predictions on test set
    test_pred_avg = np.mean(test_predictions, axis=0)
    test_pred_binary = (test_pred_avg > 0.5).astype(int).flatten()
    
    # Calculate final test metrics
    cm_test = confusion_matrix(y_test, test_pred_binary)
    
    print("\nFinal Test Set Results:")
    print(f"Accuracy: {accuracy_score(y_test, test_pred_binary):.4f}")
    print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, test_pred_binary):.4f}")
    print(f"MCC: {matthews_corrcoef(y_test, test_pred_binary):.4f}")
    print(f"Sensitivity: {cm_test[1][1]/(cm_test[1][1]+cm_test[1][0]):.4f}")
    print(f"Specificity: {cm_test[0][0]/(cm_test[0][0]+cm_test[0][1]):.4f}")
    print("Confusion Matrix:")
    print(cm_test)
    
    return model, test_pred_avg

In [8]:
# Try with 6Å threshold
model_6A, preds_6A = train_and_evaluate_combined_structure(threshold=6.0)

# Try with 8Å threshold
model_8A, preds_8A = train_and_evaluate_combined_structure(threshold=8.0)

# Try with 10Å threshold
model_10A, preds_10A = train_and_evaluate_combined_structure(threshold=10.0)

Loading structural data...
phi features shape: (8853, 2)
psi features shape: (8853, 2)
omega features shape: (8853, 2)
SASA features shape: (8853, 3)
chi1 features shape: (8853, 2)
chi2 features shape: (8853, 2)
chi3 features shape: (8853, 2)
chi4 features shape: (8853, 2)
SS features shape: (8853, 3)
plDDT features shape: (8853, 1)

Final combined features shape: (8853, 21)
Feature list lengths: [2, 2, 2, 3, 2, 2, 2, 2, 3, 1]
phi features shape: (2737, 2)
psi features shape: (2737, 2)
omega features shape: (2737, 2)
SASA features shape: (2737, 3)
chi1 features shape: (2737, 2)
chi2 features shape: (2737, 2)
chi3 features shape: (2737, 2)
chi4 features shape: (2737, 2)
SS features shape: (2737, 3)
plDDT features shape: (2737, 1)

Final combined features shape: (2737, 21)
Feature list lengths: [2, 2, 2, 3, 2, 2, 2, 2, 3, 1]


Training data shapes - Structural: (8853, 21), Contact Map: (8853, 33, 33)
Testing data shapes - Structural: (2737, 21), Contact Map: (2737, 33, 33)
Class weights: {0: 1.0388406477352734, 1: 0.9639590592334495}

Fold 1/5
Epoch 1/50


2025-05-18 12:42:16.415937: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [7082,21]
	 [[{{node Placeholder/_10}}]]
2025-05-18 12:42:16.416177: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_21' with dtype double and shape [2]
	 [[{{node Placeholder/_21}}]]
2025-05-18 12:42:16.445099: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


221/222 [============================>.] - ETA: 0s - loss: 0.7037 - accuracy: 0.5738

2025-05-18 12:42:18.586064: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1771,21]
	 [[{{node Placeholder/_10}}]]
2025-05-18 12:42:18.586246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype int64 and shape [1771]
	 [[{{node Placeholder/_12}}]]


...

222/222 [==============================] - 2s 8ms/step - loss: 0.7147 - accuracy: 0.5601 - val_loss: 0.6718 - val_accuracy: 0.5757 - lr: 0.0010
Epoch 2/50
222/222 [==============================] - 2s 7ms/step - loss: 0.6688 - accuracy: 0.5911 - val_loss: 0.6675 - val_accuracy: 0.6011 - lr: 0.0010
Epoch 3/50
222/222 [==============================] - 2s 7ms/step - loss: 0.6598 - accuracy: 0.6024 - val_loss: 0.6675 - val_accuracy: 0.5791 - lr: 0.0010
Epoch 4/50
222/222 [==============================] - 2s 7ms/step - loss: 0.6495 - accuracy: 0.6170 - val_loss: 0.6647 - val_accuracy: 0.5864 - lr: 0.0010
Epoch 5/50
222/222 [==============================] - 2s 7ms/step - loss: 0.6434 - accuracy: 0.6294 - val_loss: 0.6787 - val_accuracy: 0.5802 - lr: 0.0010
Epoch 6/50
222/222 [==============================] - 2s 7ms/step - loss: 0.6331 - accuracy: 0.6394 - val_loss: 0.6576 - val_accuracy: 0.6028 - lr: 0.0010
Epoch 7/50
222/222 [==============================] - 2s 7ms/step - loss: 0.6219 

2025-05-18 12:49:41.873278: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [1770,21]
	 [[{{node Placeholder/_10}}]]
2025-05-18 12:49:41.873434: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [1770,33,33]
	 [[{{node Placeholder/_11}}]]


52/86 [=================>............] - ETA: 0s

2025-05-18 12:49:42.133266: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [2737,33,33]
	 [[{{node Placeholder/_11}}]]
2025-05-18 12:49:42.133422: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [2737,33,33]
	 [[{{node Placeholder/_11}}]]


86/86 [==============================] - 0s 3ms/step

Fold 5 Results:
Accuracy: 0.6028
Balanced Accuracy: 0.5992
MCC: 0.2024
Sensitivity: 0.6961
Specificity: 0.5023

Average Cross-validation Results:
ACC: 0.6117 ± 0.0114
BALANCED_ACC: 0.6079 ± 0.0120
MCC: 0.2220 ± 0.0234
SN: 0.7084 ± 0.0511
SP: 0.5074 ± 0.0602

Final Test Set Results:
Accuracy: 0.4443
Balanced Accuracy: 0.5806
MCC: 0.0930
Sensitivity: 0.7458
Specificity: 0.4153
Confusion Matrix:
[[1037 1460]
 [  61  179]]
